In [1]:
import pandas as pd
import numpy as np
import os 
import sys
import glob 
import json
import importlib
import tqdm
import warnings
import matplotlib.pyplot as plt
import rapidfuzz as rf
import pickle

In [2]:
#Make sure to run this only once, to make lobbying-market the working directory
parent_dir_name=os.path.basename(os.path.dirname(os.getcwd()))
if parent_dir_name=='lobbying-market':
    os.chdir(os.path.pardir)
    print(os.getcwd())
    utils_path= os.path.abspath('utils')
    sys.path.append(utils_path)

import preprocessing as pp
importlib.reload(pp)


c:\Users\nicoj\Documents\GitHub\lobbying-market


<module 'preprocessing' from 'c:\\Users\\nicoj\\Documents\\GitHub\\lobbying-market\\utils\\preprocessing.py'>

In [3]:
lobby_by_company= pd.read_csv('LDA_data/data_by_company.csv', index_col='ticker')

In [4]:
# lobby_by_company.to_csv('LDA_data/data_by_company.csv')

In [5]:
lobby_by_company.shape

(1408, 221)

In [6]:
alist=['1','2','3','4']
qtkey_list= []
for i in range(2013,2024):
    qtkey_list=qtkey_list + [str(i)+key for key in alist]

propkey_list=['count','nan','income','expenses','total']
prop_dict={prop_key: [prop_key+'_'+ qt_key for qt_key in qtkey_list] for prop_key in propkey_list}
print(prop_dict['count'])

['count_20131', 'count_20132', 'count_20133', 'count_20134', 'count_20141', 'count_20142', 'count_20143', 'count_20144', 'count_20151', 'count_20152', 'count_20153', 'count_20154', 'count_20161', 'count_20162', 'count_20163', 'count_20164', 'count_20171', 'count_20172', 'count_20173', 'count_20174', 'count_20181', 'count_20182', 'count_20183', 'count_20184', 'count_20191', 'count_20192', 'count_20193', 'count_20194', 'count_20201', 'count_20202', 'count_20203', 'count_20204', 'count_20211', 'count_20212', 'count_20213', 'count_20214', 'count_20221', 'count_20222', 'count_20223', 'count_20224', 'count_20231', 'count_20232', 'count_20233', 'count_20234']


In [7]:
len([num for num in lobby_by_company['count_final'].values if num >0])

1408

In [8]:
qtkey_df=pd.DataFrame()
qtkey_df['qt_key']=qtkey_list
print(qtkey_df.head())
qtkey_df.set_index('qt_key',inplace=True)

  qt_key
0  20131
1  20132
2  20133
3  20134
4  20141


In [9]:
company_multi_index= pd.MultiIndex.from_product(
    [lobby_by_company.index,range(2013,2024),range(1,5)],
    names=['ticker','year','quarter']
    )
company_multi_index

MultiIndex([( 'MSFT', 2013, 1),
            ( 'MSFT', 2013, 2),
            ( 'MSFT', 2013, 3),
            ( 'MSFT', 2013, 4),
            ( 'MSFT', 2014, 1),
            ( 'MSFT', 2014, 2),
            ( 'MSFT', 2014, 3),
            ( 'MSFT', 2014, 4),
            ( 'MSFT', 2015, 1),
            ( 'MSFT', 2015, 2),
            ...
            ('VSACW', 2021, 3),
            ('VSACW', 2021, 4),
            ('VSACW', 2022, 1),
            ('VSACW', 2022, 2),
            ('VSACW', 2022, 3),
            ('VSACW', 2022, 4),
            ('VSACW', 2023, 1),
            ('VSACW', 2023, 2),
            ('VSACW', 2023, 3),
            ('VSACW', 2023, 4)],
           names=['ticker', 'year', 'quarter'], length=61952)

In [10]:
def make_df_from_ticker (ticker_key):
    multiindex= pd.MultiIndex.from_product(
    [[ticker_key],range(2013,2024),range(1,5)],
    names=['ticker','year','quarter']
    )
    result= pd.DataFrame(index=multiindex,columns= propkey_list)
    for prop_key in prop_dict:
        col_list= prop_dict[prop_key]
        value_list=lobby_by_company[prop_dict[prop_key]].loc[ticker_key].values
        result[prop_key]=value_list
    return result

In [30]:
#make DF with everything
df_lobby_all=pd.DataFrame(index=company_multi_index, columns=propkey_list)

In [13]:
df_lobby_all.loc['MSFT',:]=make_df_from_ticker('MSFT')

In [32]:
for key in lobby_by_company.index:
    df_lobby_all.loc[key,:,:]=make_df_from_ticker(key)

In [33]:
df_lobby_all.to_csv('LDA_data\data_by_company_formated.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\nicoj\AppData\Local\Temp\ipykernel_22608\1988173778.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_lobby_all.to_csv('LDA_data\data_by_company_formated.csv')


In [34]:
df_lobby_all.to_pickle('LDA_data\data_by_company_formated.pkl')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\nicoj\AppData\Local\Temp\ipykernel_22608\2802759189.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_lobby_all.to_pickle('LDA_data\data_by_company_formated.pkl')


In [40]:
assests_df= pd.read_pickle('data_processing/opensecrets_lobbying_scrape/lobbying_assets.pkl')
assests_df.set_index('ticker',inplace=True)

In [41]:
assests_df2= pd.read_pickle('data_processing/sec_assets_scrape/assets.pkl')
assests_df2.set_index('ticker',inplace=True)

In [47]:
print(assests_df.head())
print(assests_df.columns)

        level_0  index           title  \
ticker                                   
MSFT          0      0  MICROSOFT CORP   
AAPL          1      1      Apple Inc.   
NVDA          2      2     NVIDIA CORP   
GOOGL         3      3   Alphabet Inc.   
AMZN          4      4  AMAZON COM INC   

                                                   assets  \
ticker                                                      
MSFT    {'label': 'Assets', 'description': 'Sum of the...   
AAPL    {'label': 'Assets', 'description': 'Sum of the...   
NVDA    {'label': 'Assets', 'description': 'Sum of the...   
GOOGL   {'label': 'Assets', 'description': 'Sum of the...   
AMZN    {'label': 'Assets', 'description': 'Sum of the...   

                        sector                        industry  \
ticker                                                           
MSFT                Technology         Software—Infrastructure   
AAPL                Technology            Consumer Electronics   
NVDA         

In [42]:
def get_last_number (aux_str):
    return int(str(aux_str)[-1:])

def index_merged_summary(ticker_key):
    indexed_df=assests_df.loc[ticker_key,'merged_summary']
    indexed_df['ticker']=ticker_key
    indexed_df['sector']=assests_df.loc[ticker_key,'sector']
    indexed_df['industry']=assests_df.loc[ticker_key,'industry']
    indexed_df['os_orgid']=assests_df.loc[ticker_key,'os_orgid']
    indexed_df= indexed_df.rename(columns={'quarter': 'qtstr'})
    indexed_df['quarter']=indexed_df['qtstr'].apply(get_last_number)
    return indexed_df.set_index(['ticker','year','quarter'])

def index_assets_summary(ticker_key):
    indexed_df=assests_df2.loc[ticker_key,'assets_summary']
    indexed_df['ticker']=ticker_key
    indexed_df['sector']=assests_df2.loc[ticker_key,'sector']
    indexed_df['industry']=assests_df2.loc[ticker_key,'industry']
    indexed_df['year']=indexed_df['year'].astype(int)
    indexed_df= indexed_df.rename(columns={'quarter': 'qtstr'})
    indexed_df['quarter']=indexed_df['qtstr'].apply(get_last_number)
    return indexed_df.set_index(['ticker','year','quarter'])

In [43]:
def get_combined_data_by_ticker(ticker_key):
    temp_df1 = make_df_from_ticker(ticker_key)
    temp_df2 = index_assets_summary(ticker_key)
    merged_df= temp_df1.merge(temp_df2, left_index=True, right_index=True, how='outer')
    merged_df['sector']=assests_df2.loc[ticker_key,'sector']
    merged_df['industry']=assests_df2.loc[ticker_key,'industry']
    return merged_df

In [49]:
index_merged_summary('MSFT').head(10)

end           val qtstr  lobbying_costs  \
ticker year quarter                                                   
MSFT   2009 2        2009-06-30   77888000000    Q2       1890000.0   
            3        2009-09-30   81612000000    Q3       1490000.0   
            4        2009-12-31   82096000000    Q4       1690000.0   
       2010 1        2010-03-31   84910000000    Q1       1720000.0   
            2        2010-06-30   86113000000    Q2       1850000.0   
            3        2010-09-30   91540000000    Q3       1630000.0   
            4        2010-12-31   92306000000    Q4       1710000.0   
       2011 1        2011-03-31   99727000000    Q1       1720000.0   
            2        2011-06-30  108704000000    Q2       1852000.0   
            3        2011-09-30  107415000000    Q3       1883000.0   

                         sector                 industry    os_orgid  
ticker year quarter                                                   
MSFT   2009 2        Technology  Software—Infrastructure  D000000115  
            3        Technology  Software—Infrastructure  D000000115  
            4        Technology  Software—Infrastructure  D000000115  
       2010 1        Technology  Software—Infrastructure  D000000115  
            2        Technology  Software—Infrastructure  D000000115  
            3        Technology  Software—Infrastructure  D000000115  
            4        Technology  Software—Infrastructure  D000000115  
       2011 1        Technology  Software—Infrastructure  D000000115  
            2        Technology  Software—Infrastructure  D000000115  
            3        Technology  Software—Infrastructure  D000000115

In [54]:
indexed_assets_df=pd.DataFrame()
for key in assests_df.index:
    indexed_assets_df=pd.concat([indexed_assets_df,index_merged_summary(key)])

In [55]:
indexed_assets_df.to_csv('data_processing/opensecrets_lobbying_scrape/indexed_assets_df.csv')

In [56]:
indexed_assets_df.to_pickle('data_processing/opensecrets_lobbying_scrape/indexed_assets_df.pkl')

In [58]:
df_lobby_all=pd.read_pickle('LDA_data/data_by_company_formated.pkl')

In [62]:
df1= get.head(30)
df2= indexed_assets_df.head(30)
df1.merge(df2, left_index=True, right_index=True, how='outer')

count  nan     income   expenses      total         end  \
ticker year quarter                                                           
MSFT   2009 2         NaN  NaN        NaN        NaN        NaN  2009-06-30   
            3         NaN  NaN        NaN        NaN        NaN  2009-09-30   
            4         NaN  NaN        NaN        NaN        NaN  2009-12-31   
       2010 1         NaN  NaN        NaN        NaN        NaN  2010-03-31   
            2         NaN  NaN        NaN        NaN        NaN  2010-06-30   
            3         NaN  NaN        NaN        NaN        NaN  2010-09-30   
            4         NaN  NaN        NaN        NaN        NaN  2010-12-31   
       2011 1         NaN  NaN        NaN        NaN        NaN  2011-03-31   
            2         NaN  NaN        NaN        NaN        NaN  2011-06-30   
            3         NaN  NaN        NaN        NaN        NaN  2011-09-30   
            4         NaN  NaN        NaN        NaN        NaN  2011-12-31   
       2012 1         NaN  NaN        NaN        NaN        NaN  2012-03-31   
            2         NaN  NaN        NaN        NaN        NaN  2012-06-30   
            3         NaN  NaN        NaN        NaN        NaN  2012-09-30   
            4         NaN  NaN        NaN        NaN        NaN  2012-12-31   
       2013 1        23.0  0.0  1057500.0  2530000.0  3587500.0  2013-03-31   
            2        23.0  0.0   980000.0  2960000.0  3940000.0  2013-06-30   
            3        23.0  0.0   987000.0  2230000.0  3217000.0  2013-09-30   
            4        24.0  0.0  1070000.0  2770000.0  3840000.0  2013-12-31   
       2014 1        22.0  0.0   937000.0  2080000.0  3017000.0  2014-03-31   
            2        22.0  0.0   987000.0  2340000.0  3327000.0  2014-06-30   
            3        22.0  0.0   855000.0  1660000.0  2515000.0  2014-09-30   
            4        21.0  0.0   805000.0  2250000.0  3055000.0  2014-12-31   
       2015 1        23.0  0.0   865000.0  1890000.0  2755000.0  2015-03-31   
            2        22.0  0.0   895000.0  2240000.0  3135000.0  2015-06-30   
            3        22.0  0.0   960000.0  1890000.0  2850000.0  2015-09-30   
            4        22.0  0.0   940000.0  2470000.0  3410000.0  2015-12-31   
       2016 1        24.0  1.0   960000.0  2020000.0  2980000.0  2016-03-31   
            2        25.0  0.0  1040000.0  2070000.0  3110000.0  2016-06-30   
            3        26.0  0.0  1050000.0  2220000.0  3270000.0  2016-09-30   
            4        22.0  0.0   920000.0  2400000.0  3320000.0         NaN   
       2017 1        21.0  0.0   985000.0  2300000.0  3285000.0         NaN   
            2        22.0  0.0  1045000.0  2070000.0  3115000.0         NaN   
            3        22.0  0.0  1045000.0  1950000.0  2995000.0         NaN   
            4        21.0  0.0  1020000.0  2180000.0  3200000.0         NaN   
       2018 1        22.0  0.0  1140000.0  2300000.0  3440000.0         NaN   
            2        21.0  0.0   950000.0  2640000.0  3590000.0         NaN   
            3        21.0  0.0   950000.0  2240000.0  3190000.0         NaN   
            4        21.0  0.0   970000.0  2340000.0  3310000.0         NaN   
       2019 1        20.0  1.0   855000.0  2790000.0  3645000.0         NaN   
            2        22.0  0.0   925000.0  2740000.0  3665000.0         NaN   
            3        21.0  0.0   970000.0  2250000.0  3220000.0         NaN   
            4        22.0  0.0   970000.0  2390000.0  3360000.0         NaN   
       2020 1        22.0  0.0   990000.0  2394000.0  3384000.0         NaN   
            2        22.0  0.0   990000.0  2910000.0  3900000.0         NaN   

                              val qtstr  lobbying_costs      sector  \
ticker year quarter                                                   
MSFT   2009 2        7.788800e+10    Q2       1890000.0  Technology   
            3        8.161200e+10    Q3       1490000.0  Technology   
            4        8

In [68]:
get_combined_data_by_ticker('MSFT')

count  nan     income   expenses      total         end  \
ticker year quarter                                                            
MSFT   2009 2          NaN  NaN        NaN        NaN        NaN  2009-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2009-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2009-12-31   
       2010 1          NaN  NaN        NaN        NaN        NaN  2010-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2010-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2010-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2010-12-31   
       2011 1          NaN  NaN        NaN        NaN        NaN  2011-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2011-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2011-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2011-12-31   
       2012 1          NaN  NaN        NaN        NaN        NaN  2012-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2012-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2012-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2012-12-31   
       2013 1         23.0  0.0  1057500.0  2530000.0  3587500.0  2013-03-31   
            2         23.0  0.0   980000.0  2960000.0  3940000.0  2013-06-30   
            3         23.0  0.0   987000.0  2230000.0  3217000.0  2013-09-30   
            4         24.0  0.0  1070000.0  2770000.0  3840000.0  2013-12-31   
       2014 1         22.0  0.0   937000.0  2080000.0  3017000.0  2014-03-31   
            2         22.0  0.0   987000.0  2340000.0  3327000.0  2014-06-30   
            3         22.0  0.0   855000.0  1660000.0  2515000.0  2014-09-30   
            4         21.0  0.0   805000.0  2250000.0  3055000.0  2014-12-31   
       2015 1         23.0  0.0   865000.0  1890000.0  2755000.0  2015-03-31   
            2         22.0  0.0   895000.0  2240000.0  3135000.0  2015-06-30   
            3         22.0  0.0   960000.0  1890000.0  2850000.0  2015-09-30   
            4         22.0  0.0   940000.0  2470000.0  3410000.0  2015-12-31   
       2016 1         24.0  1.0   960000.0  2020000.0  2980000.0  2016-03-31   
            2         25.0  0.0  1040000.0  2070000.0  3110000.0  2016-06-30   
            3         26.0  0.0  1050000.0  2220000.0  3270000.0  2016-09-30   
            4         22.0  0.0   920000.0  2400000.0  3320000.0  2016-12-31   
       2017 1         21.0  0.0   985000.0  2300000.0  3285000.0  2017-03-31   
            2         22.0  0.0  1045000.0  2070000.0  3115000.0  2017-06-30   
            3         22.0  0.0  1045000.0  1950000.0  2995000.0  2017-09-30   
            4         21.0  0.0  1020000.0  2180000.0  3200000.0  2017-12-31   
       2018 1         22.0  0.0  1140000.0  2300000.0  3440000.0  2018-03-31   
            2         21.0  0.0   950000.0  2640000.0  3590000.0  2018-06-30   
            3         21.0  0.0   950000.0  2240000.0  3190000.0  2018-09-30   
            4         21.0  0.0   970000.0  2340000.0  3310000.0  2018-12-31   
       2019 1         20.0  1.0   855000.0  2790000.0  3645000.0  2019-03-31   
            2         22.0  0.0   925000.0  2740000.0  3665000.0  2019-06-30   
            3         21.0  0.0   970000.0  2250000.0  3220000.0  2019-09-30   
            4         22.0  0.0   970000.0  2390000.0  3360000.0  2019-12-31   
       2020 1         22.0  0.0   990000.0  2394000.0  3384000.0  2020-03-31   
            2         22.0  0.0   990000.0  2910000.0  3900000.0  2020-06-30   
            3         21.0  1.0   940000.0  1880000.0  2820000.0  2020-09-30   
            4         21.0  0.0   965000.0  2190000.0  3155000.0  2020-12-31   
       2021 1         23.0  0.0  1050000.0  2590000.0  3640000.0  2021-03-31   
            2        

In [23]:
pd.Series(lobby_by_company.index).isin(assests_df2.index).value_counts()

ticker
True    1408
Name: count, dtype: int64

In [21]:
pd.Series(assests_df2.index).isin(lobby_by_company.index).value_counts()

ticker
True     1408
False    1179
Name: count, dtype: int64

In [11]:
df2=pd.read_pickle('data_processing/sec_assets_scrape/assets.pkl')

In [12]:
df2.shape

(2587, 6)

In [13]:
df2.head()

,ticker,title,assets,sector,industry,assets_summary
0,MSFT,MICROSOFT CORP,"{'label': 'Assets', 'description': 'Sum of the...",Technology,Software—Infrastructure,end val quarter year 0 ...
1,AAPL,Apple Inc.,"{'label': 'Assets', 'description': 'Sum of the...",Technology,Consumer Electronics,end val quarter year 0 ...
2,NVDA,NVIDIA CORP,"{'label': 'Assets', 'description': 'Sum of the...",Technology,Semiconductors,end val quarter year 0 ...
3,GOOGL,Alphabet Inc.,"{'label': 'Assets', 'description': 'Sum of the...",Communication Services,Internet Content & Information,end val quarter year 0 ...
4,AMZN,AMAZON COM INC,"{'label': 'Assets', 'description': 'Sum of the...",Consumer Cyclical,Internet Retail,end val quarter year 0 ...


In [25]:
index_assets_summary('MSFT')

end           val qtstr      sector  \
ticker year quarter                                               
MSFT   2009 2        2009-06-30   77888000000    Q2  Technology   
            3        2009-09-30   81612000000    Q3  Technology   
            4        2009-12-31   82096000000    Q4  Technology   
       2010 1        2010-03-31   84910000000    Q1  Technology   
            2        2010-06-30   86113000000    Q2  Technology   
            3        2010-09-30   91540000000    Q3  Technology   
            4        2010-12-31   92306000000    Q4  Technology   
       2011 1        2011-03-31   99727000000    Q1  Technology   
            2        2011-06-30  108704000000    Q2  Technology   
            3        2011-09-30  107415000000    Q3  Technology   
            4        2011-12-31  112243000000    Q4  Technology   
       2012 1        2012-03-31  118010000000    Q1  Technology   
            2        2012-06-30  121271000000    Q2  Technology   
            3        2012-09-30  121876000000    Q3  Technology   
            4        2012-12-31  128683000000    Q4  Technology   
       2013 1        2013-03-31  134105000000    Q1  Technology   
            2        2013-06-30  142431000000    Q2  Technology   
            3        2013-09-30  142348000000    Q3  Technology   
            4        2013-12-31  153543000000    Q4  Technology   
       2014 1        2014-03-31  156119000000    Q1  Technology   
            2        2014-06-30  172384000000    Q2  Technology   
            3        2014-09-30  169656000000    Q3  Technology   
            4        2014-12-31  174848000000    Q4  Technology   
       2015 1        2015-03-31  176683000000    Q1  Technology   
            2        2015-06-30  176223000000    Q2  Technology   
            3        2015-09-30  172896000000    Q3  Technology   
            4        2015-12-31  180098000000    Q4  Technology   
       2016 1        2016-03-31  181869000000    Q1  Technology   
            2        2016-06-30  193694000000    Q2  Technology   
            3        2016-09-30  212524000000    Q3  Technology   
            4        2016-12-31  224610000000    Q4  Technology   
       2017 1        2017-03-31  225017000000    Q1  Technology   
            2        2017-06-30  241086000000    Q2  Technology   
            3        2017-09-30  249097000000    Q3  Technology   
            4        2017-12-31  256003000000    Q4  Technology   
       2018 1        2018-03-31  245497000000    Q1  Technology   
            2        2018-06-30  258848000000    Q2  Technology   
            3        2018-09-30  257619000000    Q3  Technology   
            4        2018-12-31  258859000000    Q4  Technology   
       2019 1        2019-03-31  263281000000    Q1  Technology   
            2        2019-06-30  286556000000    Q2  Technology   
            3        2019-09-30  278955000000    Q3  Technology   
            4        2019-12-31  282794000000    Q4  Technology   
       2020 1        2020-03-31  285449000000    Q1  Technology   
            2        2020-06-30  301311000000    Q2  Technology   
            3        2020-09-30  301001000000    Q3  Technology   
            4        2020-12-31  304137000000    Q4  Technology   
       2021 1        2021-03-31  308879000000    Q1  Technology   
            2        2021-06-30  333779000000    Q2  Technology   
            3        2021-09-30  335418000000    Q3  Technology   
            4        2021-12-31  340389000000    Q4  Technology   
       2022 1        2022-03-31  344607000000    Q1  Technology   
            2        2022-06-30  364840000000    Q2  Technology   
            3        2022-09-30  359784000000    Q3  Technology   
            4        2022-12-31  364552000000    Q4  Technology   
       2023 1        2023-03-31  380088000000    Q1  Technology   
            2        2023-06-30  411976000000    Q2  Technology   
            3        2023-09-30  445785000000    Q3  Technology   
        

In [29]:
get_combined_data_by_ticker('MSFT')

count  nan     income   expenses      total         end  \
ticker year quarter                                                            
MSFT   2009 2          NaN  NaN        NaN        NaN        NaN  2009-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2009-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2009-12-31   
       2010 1          NaN  NaN        NaN        NaN        NaN  2010-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2010-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2010-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2010-12-31   
       2011 1          NaN  NaN        NaN        NaN        NaN  2011-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2011-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2011-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2011-12-31   
       2012 1          NaN  NaN        NaN        NaN        NaN  2012-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2012-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2012-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2012-12-31   
       2013 1         23.0  0.0  1057500.0  2530000.0  3587500.0  2013-03-31   
            2         23.0  0.0   980000.0  2960000.0  3940000.0  2013-06-30   
            3         23.0  0.0   987000.0  2230000.0  3217000.0  2013-09-30   
            4         24.0  0.0  1070000.0  2770000.0  3840000.0  2013-12-31   
       2014 1         22.0  0.0   937000.0  2080000.0  3017000.0  2014-03-31   
            2         22.0  0.0   987000.0  2340000.0  3327000.0  2014-06-30   
            3         22.0  0.0   855000.0  1660000.0  2515000.0  2014-09-30   
            4         21.0  0.0   805000.0  2250000.0  3055000.0  2014-12-31   
       2015 1         23.0  0.0   865000.0  1890000.0  2755000.0  2015-03-31   
            2         22.0  0.0   895000.0  2240000.0  3135000.0  2015-06-30   
            3         22.0  0.0   960000.0  1890000.0  2850000.0  2015-09-30   
            4         22.0  0.0   940000.0  2470000.0  3410000.0  2015-12-31   
       2016 1         24.0  1.0   960000.0  2020000.0  2980000.0  2016-03-31   
            2         25.0  0.0  1040000.0  2070000.0  3110000.0  2016-06-30   
            3         26.0  0.0  1050000.0  2220000.0  3270000.0  2016-09-30   
            4         22.0  0.0   920000.0  2400000.0  3320000.0  2016-12-31   
       2017 1         21.0  0.0   985000.0  2300000.0  3285000.0  2017-03-31   
            2         22.0  0.0  1045000.0  2070000.0  3115000.0  2017-06-30   
            3         22.0  0.0  1045000.0  1950000.0  2995000.0  2017-09-30   
            4         21.0  0.0  1020000.0  2180000.0  3200000.0  2017-12-31   
       2018 1         22.0  0.0  1140000.0  2300000.0  3440000.0  2018-03-31   
            2         21.0  0.0   950000.0  2640000.0  3590000.0  2018-06-30   
            3         21.0  0.0   950000.0  2240000.0  3190000.0  2018-09-30   
            4         21.0  0.0   970000.0  2340000.0  3310000.0  2018-12-31   
       2019 1         20.0  1.0   855000.0  2790000.0  3645000.0  2019-03-31   
            2         22.0  0.0   925000.0  2740000.0  3665000.0  2019-06-30   
            3         21.0  0.0   970000.0  2250000.0  3220000.0  2019-09-30   
            4         22.0  0.0   970000.0  2390000.0  3360000.0  2019-12-31   
       2020 1         22.0  0.0   990000.0  2394000.0  3384000.0  2020-03-31   
            2         22.0  0.0   990000.0  2910000.0  3900000.0  2020-06-30   
            3         21.0  1.0   940000.0  1880000.0  2820000.0  2020-09-30   
            4         21.0  0.0   965000.0  2190000.0  3155000.0  2020-12-31   
       2021 1         23.0  0.0  1050000.0  2590000.0  3640000.0  2021-03-31   
            2        

In [28]:
make_df_from_ticker('MSFT').head(30).merge(index_assets_summary('MSFT').head(30), left_index=True, right_index=True, how='outer')

count  nan     income   expenses      total         end  \
ticker year quarter                                                            
MSFT   2009 2          NaN  NaN        NaN        NaN        NaN  2009-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2009-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2009-12-31   
       2010 1          NaN  NaN        NaN        NaN        NaN  2010-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2010-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2010-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2010-12-31   
       2011 1          NaN  NaN        NaN        NaN        NaN  2011-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2011-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2011-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2011-12-31   
       2012 1          NaN  NaN        NaN        NaN        NaN  2012-03-31   
            2          NaN  NaN        NaN        NaN        NaN  2012-06-30   
            3          NaN  NaN        NaN        NaN        NaN  2012-09-30   
            4          NaN  NaN        NaN        NaN        NaN  2012-12-31   
       2013 1         23.0  0.0  1057500.0  2530000.0  3587500.0  2013-03-31   
            2         23.0  0.0   980000.0  2960000.0  3940000.0  2013-06-30   
            3         23.0  0.0   987000.0  2230000.0  3217000.0  2013-09-30   
            4         24.0  0.0  1070000.0  2770000.0  3840000.0  2013-12-31   
       2014 1         22.0  0.0   937000.0  2080000.0  3017000.0  2014-03-31   
            2         22.0  0.0   987000.0  2340000.0  3327000.0  2014-06-30   
            3         22.0  0.0   855000.0  1660000.0  2515000.0  2014-09-30   
            4         21.0  0.0   805000.0  2250000.0  3055000.0  2014-12-31   
       2015 1         23.0  0.0   865000.0  1890000.0  2755000.0  2015-03-31   
            2         22.0  0.0   895000.0  2240000.0  3135000.0  2015-06-30   
            3         22.0  0.0   960000.0  1890000.0  2850000.0  2015-09-30   
            4         22.0  0.0   940000.0  2470000.0  3410000.0  2015-12-31   
       2016 1         24.0  1.0   960000.0  2020000.0  2980000.0  2016-03-31   
            2         25.0  0.0  1040000.0  2070000.0  3110000.0  2016-06-30   
            3         26.0  0.0  1050000.0  2220000.0  3270000.0  2016-09-30   
            4         22.0  0.0   920000.0  2400000.0  3320000.0         NaN   
       2017 1         21.0  0.0   985000.0  2300000.0  3285000.0         NaN   
            2         22.0  0.0  1045000.0  2070000.0  3115000.0         NaN   
            3         22.0  0.0  1045000.0  1950000.0  2995000.0         NaN   
            4         21.0  0.0  1020000.0  2180000.0  3200000.0         NaN   
       2018 1         22.0  0.0  1140000.0  2300000.0  3440000.0         NaN   
            2         21.0  0.0   950000.0  2640000.0  3590000.0         NaN   
            3         21.0  0.0   950000.0  2240000.0  3190000.0         NaN   
            4         21.0  0.0   970000.0  2340000.0  3310000.0         NaN   
       2019 1         20.0  1.0   855000.0  2790000.0  3645000.0         NaN   
            2         22.0  0.0   925000.0  2740000.0  3665000.0         NaN   
            3         21.0  0.0   970000.0  2250000.0  3220000.0         NaN   
            4         22.0  0.0   970000.0  2390000.0  3360000.0         NaN   
       2020 1         22.0  0.0   990000.0  2394000.0  3384000.0         NaN   
            2         22.0  0.0   990000.0  2910000.0  3900000.0         NaN   

                              val qtstr      sector                 industry  
ticker year quarter                                                           
MSFT   2009 2        7.788800e+10    Q2  Technology  Software—Infrastructure  
            3        8.

In [35]:
all_data_merged_df=pd.DataFrame()
for key in lobby_by_company.index:
    all_data_merged_df=pd.concat([all_data_merged_df,get_combined_data_by_ticker(key)])

In [37]:
all_data_merged_df.to_pickle('LDA_data/all_data_merged_df.pkl')

In [11]:
all_data_merged_df=pd.read_pickle('LDA_data/all_data_merged_df.pkl')

In [12]:
filtered_merged_data=all_data_merged_df[all_data_merged_df['qtstr'].notna()]
filtered_merged_data=filtered_merged_data[filtered_merged_data['count'].notna()]

In [13]:
filtered_merged_data.shape

(45722, 10)

In [14]:
filtered_merged_data.loc['MSFT',:,:].shape[0]

44

In [18]:
aux_ticker_list=[]
aux_ticker_count={}
for key in lobby_by_company.index:
    aux_filter_data=all_data_merged_df.loc[key,:,:]
    aux_filter_data=aux_filter_data[aux_filter_data['qtstr'].notna()]
    aux_filter_data=aux_filter_data[aux_filter_data['count'].notna()]
    aux_count=len(aux_filter_data)
    if aux_count>0: aux_ticker_list.append(key)
    aux_ticker_count[key]=aux_count

In [19]:
len(aux_ticker_list)

1407

In [20]:
series1=pd.Series(aux_ticker_count.values())

In [21]:
series1.value_counts()

44    624
13     62
12     47
43     43
15     38
45     37
9      29
14     29
41     28
11     25
24     23
20     20
16     20
21     20
25     19
42     18
17     17
19     17
10     16
39     15
23     15
29     14
35     14
7      14
37     14
28     13
8      13
27     13
40     13
36     12
5      12
3      11
32     11
46     11
18     11
26     10
31     10
4      10
38      9
33      9
6       6
22      5
30      5
2       2
34      2
48      1
0       1
Name: count, dtype: int64

In [50]:
len(number_of_entries_dict)

1408

In [35]:
filter_threshold=[key for key in aux_ticker_count if aux_ticker_count[key]>30]

In [45]:
all_data_over_threshold=pd.DataFrame()
for key in filter_threshold:
    all_data_over_threshold= pd.concat([all_data_over_threshold, get_combined_data_by_ticker(key)])
    
all_data_over_threshold=all_data_over_threshold[all_data_over_threshold['qtstr'].notna()]
all_data_over_threshold=all_data_over_threshold[all_data_over_threshold['count'].notna()]

In [6]:
all_data_over_threshold=all_data_over_threshold[all_data_over_threshold['qtstr'].notna()]
all_data_over_threshold=all_data_over_threshold[all_data_over_threshold['count'].notna()]

In [3]:
all_data_over_threshold.to_pickle('LDA_data/all_data_over_threshold.pkl')

NameError: name 'all_data_over_threshold' is not defined

In [4]:
all_data_over_threshold= pd.read_pickle('LDA_data/all_data_over_threshold.pkl')

In [7]:
all_data_over_threshold.head()

count  nan     income   expenses      total         end  \
ticker year quarter                                                            
MSFT   2013 1         23.0  0.0  1057500.0  2530000.0  3587500.0  2013-03-31   
            2         23.0  0.0   980000.0  2960000.0  3940000.0  2013-06-30   
            3         23.0  0.0   987000.0  2230000.0  3217000.0  2013-09-30   
            4         24.0  0.0  1070000.0  2770000.0  3840000.0  2013-12-31   
       2014 1         22.0  0.0   937000.0  2080000.0  3017000.0  2014-03-31   

                              val qtstr      sector                 industry  
ticker year quarter                                                           
MSFT   2013 1        1.341050e+11    Q1  Technology  Software—Infrastructure  
            2        1.424310e+11    Q2  Technology  Software—Infrastructure  
            3        1.423480e+11    Q3  Technology  Software—Infrastructure  
            4        1.535430e+11    Q4  Technology  Software—Infrastructure  
       2014 1        1.561190e+11    Q1  Technology  Software—Infrastructure